This is an example using [rdflib](https://rdflib.readthedocs.io/), which is a Python Library that implements reading and writing RDF and SPARQL queries.

# rdflib examples
These are the most basic translations of questions into SPARQL queries.

## Permitted Uses

In [1]:
import rdflib
g_uses = rdflib.Graph()
# load the graph related to the permitted uses
g_uses.parse("permits_use2.ttl")

<Graph identifier=N9454e6f2c3114f73bd4fc270bb1327a2 (<class 'rdflib.graph.Graph'>)>

In [2]:
# number of triples in the graph?
len(g_uses)

228

### 1. Query for "Which zoning districts allow a recreation center?"

In [3]:

sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?zoning_label
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :permitsUse "recreation centers" .
        ?zoning rdfs:label ?zoning_label .
}
"""

results = g_uses.query(sparql_query)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

Zoning District (Label): R1
Zoning District (Label): R2
Zoning District (Label): R3


### 2. Query for "Which zoning districts permit duplexes?"
These are listed as "Two-family dwellings"

In [4]:
sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?zoning_label
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :permitsUse "Two-family dwellings" .
        ?zoning rdfs:label ?zoning_label .
}
"""

results = g_uses.query(sparql_query)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

Zoning District (Label): R2
Zoning District (Label): R3


### 3. Which zoning district allows restaurants?

In [5]:
sparql_query = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?zoning_label
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :permitsUse "restaurants" .
        ?zoning rdfs:label ?zoning_label .
}
"""

results = g_uses.query(sparql_query)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

Zoning District (Label): C1
Zoning District (Label): C2
Zoning District (Label): C3
Zoning District (Label): C4


### 4. Are automotive sales allowed in a C-3 zoning district?

In [42]:
# This is more litterly ASKing if there is a triple (or set of triples existing in the dataset)
# matching atleast once.

sparql_query_auto_sales_ask1 = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

ASK {
		:c3 :permitsUse "automotive sales" .
}
"""

results = g_uses.query(sparql_query_auto_sales_ask1)

results.askAnswer


True

In [43]:
sparql_query_auto_sales_ask2 = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

ASK {
		?zoning :permitsUse "automotive sales" ;
                rdfs:label "C3" .
}
"""

results = g_uses.query(sparql_query_auto_sales_ask2)

results.askAnswer

True

### 5. I would like to build an indoor theater.  Which zoning districts permit that? 

In [2]:
sparql_query_theater = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?zoning_label
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :permitsUse "indoor theaters" .
        ?zoning rdfs:label ?zoning_label .
}
"""

results = g_uses.query(sparql_query_theater)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

NameError: name 'g_uses' is not defined

##  "Bulk" Dimensional Requirements

In [62]:
import rdflib
g_bulk = rdflib.Graph()
# load the graph related to the permitted uses
g_bulk.parse("bulk.ttl")

# this is the version with units
g_bulk_units = rdflib.Graph()
# load the graph related to the permitted uses
g_bulk_units.parse("bulk2.ttl")

<Graph identifier=N764b49d2906e430b9fb7ff408bbf3271 (<class 'rdflib.graph.Graph'>)>

In [3]:
# number of triples in the graph?
len(g_bulk)

151

### 6. What is the minimum lot size in the R1a zoning district?

In [8]:
# This isn't how you would want to interact with this.
sparql_query_min_lot_terse = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?min_lot_size

WHERE {
		:r1a :minLotSize ?min_lot_size .
}
"""
results_min_lot1 = g_bulk.query(sparql_query_min_lot_terse)

for row in results_min_lot1:
    print(row.min_lot_size)

35000


In [10]:
sparql_query_min_lot = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?min_lot_size
# ?zoning -- this could also be exposed, if needed

WHERE {
		?zoning :minLotSize ?min_lot_size ;
                rdfs:label "R1a" .
}
"""

results = g_bulk.query(sparql_query_min_lot)

for row in results:
    print(row.min_lot_size)
    # print(f'Zoning District (Label): {row.zoning_label}')

35000


In [18]:
for row in results:
    print(row)

(rdflib.term.Literal('35000', datatype=rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#integer')),)


### 6b. Is the minimum lot size for property in the R1a zoning district 35,000 square feet?

In [38]:
sparql_query_min_lot_ask ="""
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>

ASK {
		?zoning :minLotSize 35000 ;
                rdfs:label "R1a" .
}
"""

results = g_bulk.query(sparql_query_min_lot_ask)
results.askAnswer

True

### 7. What is the minimum lot width for the R1c Zoning District? 

In [11]:
# terse query, not as useful
query_min_lot_width_terse = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?lot_width

WHERE {
	:r1c :minLotWidth ?lot_width .
}
"""

results = g_bulk.query(query_min_lot_width_terse)

for row in results:
    print(row.lot_width)

75


In [13]:
# query based on zoning label "R1c"
query_min_lot_width = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?lot_width

WHERE {
	?zoning :minLotWidth ?lot_width ;
            rdfs:label   "R1c" .
}
"""

results = g_bulk.query(query_min_lot_width)

for row in results:
    print(row.lot_width)

75


### 8. What is the maximum building height in the FI2 District?

In [15]:
# query based on zoning label "FI2"
query_max_building_height = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?max_building_height

WHERE {
	?zoning :maxBuildingHeight ?max_building_height ;
            rdfs:label   "FI2" .
}
"""

results = g_bulk.query(query_max_building_height)

for row in results:
    print(row.max_building_height)

80


In [18]:
# same query as above querying the version with units
results = g_bulk_units.query(query_max_building_height)

for row in results:
    print(row.max_building_height)

80 [ft_i]


### 9. What is the minimum front setback in the R3b zoning district? 

In [10]:
query_min_front_setback = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?min_front_setback

WHERE {
	?zoning :minFrontSetback ?min_front_setback ;
            rdfs:label   "R3b" .
}
"""

results = g_bulk_units.query(query_min_front_setback)

for row in results:
    print(row.min_front_setback)

NameError: name 'g_bulk_units' is not defined

### 10. What is the minimum lot depth in an A1 zoning district?

In [9]:
query_min_lot_depth = """
PREFIX : <http://www.example.org/ns/lu/zoning#>

SELECT ?min_lot_depth

WHERE {
	?zoning :minLotDepth ?min_lot_depth ;
            rdfs:label   "A1" .
}
"""

results = g_bulk_units.query(query_min_lot_depth)

for row in results:
    print(row.min_lot_depth)

NameError: name 'g_bulk_units' is not defined

# Starting Templates
Use python's [string.Template](https://docs.python.org/3.4/library/string.html#template-strings) to create templates of the above questions.  It is a simple template format that uses a dollar sign `$` to denote a substitution.

## Starting Templates - Permitted Uses

### Template 1 - template_use_1var_m_answer
Covers questions #1-3 and 5

In [57]:
from string import Template

prefix_header = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>
"""

name_template1 = "template_use_1var_m_answer"

# May need words like for/a/an to make these questions proper English. 
text_templates1 = ["Which zoning districts allow ${use}?", 
                   "Which zoning districts permit ${use}?", 
                   "I would like to build ${use}.  Which zoning districts permits this use?"]
sparql_template1 = """
SELECT ?zoning_label

WHERE {
		?zoning :permitsUse "${use}" .
        ?zoning rdfs:label ?zoning_label .
}
"""

uses_t1 = ["recreation centers", "restaurants", "indoor theaters"]

# expectated output from SPARQL: list of zoning districts

In [1]:
# for text_template in text_template:
#    for use in uses_template:
#        Template(text_template, use=use)
question_obj = Template(text_templates1[0])
q_text = question_obj.substitute(use=uses_t1[1])
# sparql_template1    
print(f"Question: {q_text}")

sparql_text_obj = Template(sparql_template1)
sparql_query_text = sparql_text_obj.substitute(use=uses_t1[1])
print(f"SPARQL:")
print(sparql_query_text)

NameError: name 'Template' is not defined

In [13]:
print('Results: ')
results = g_uses.query(sparql_query_text)

for row in results:
    print(f'Zoning District (Label): {row.zoning_label}')

Results: 
Zoning District (Label): C1
Zoning District (Label): C2
Zoning District (Label): C3
Zoning District (Label): C4


### Template 2 - template_use_2var_yn_answer
Covers question #4

In [54]:

name_template2 = "template_use_2var_yn_answer"

# Are/Is,  the uses in IZC are plural.  Typically someone asks about a 
# non-residential uses in the singular.
text_templates2 = ["Are $use allowed in a $zoning zoning district?",
                  "Are $use permitted in a $zoning zoning district?"]


prefix_header = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>
"""

sparql_template2 = """
ASK {
		?zoning :permitsUse "${use}" ;
                rdfs:label "${zoning}" .
}
"""

use_t2 = ["automotive sales"]
zoning_t2 = ["C3"]

# expectated output from SPARQL: boolean (True/False)

### Template 5 - template_use_1var_yn_answer

In [4]:

name_template5 = "template_use_1var_yn_answer"

text_templates5 = ["Are $use permitted?",
                  "Are $use allowed?"]

prefix_header = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX : <http://www.example.org/ns/lu/zoning#>
"""

sparql_template5 = """
ASK {
		?zoning :permitsUse "${use}" .
}
"""

# expectated output from SPARQL: boolean (True/False)

## Starting Templates - Dimensional Requirements

### Template 3 - template_dimreg_2var_m_answer
Covers Question 6, 7, 8, 9, 10

In [59]:
name_template3 = "template_dimreg_2var_m_answer"

text_templates3 = ["What is the $regulation in the $zoning zoning district?"]

# regulations's description is the key, the RDF name is the value
regulation_dict_t3 = {"minimum lot size":":minLotSize",
                      "minimum lot width":":minLotWidth", 
                      "maximum building height":":maxBuildingHeight",
                      "minimum front setback":":minFrontSetback",
                      "minimum lot depth": ":minLotDepth"}

zoning_t3 = ["R1a", "R1c", "FI2", "R3b", "A1"]

# regulation_value is a generic name, not really a technical term
sparql_template3 ="""
SELECT ?regulation_value

WHERE {
		?zoning $regulation ?regulation_value ;
                rdfs:label "${zoning}" .
}
"""
# expectated output from SPARQL: value for the regulation and optionally a unit 

In [ ]:
def generate_question(q_template, d_vals):
    Template(q_template).substitute(d_vals)

### Template 4 - template_dimreg_4var_yn_answer
Covers question 6b

In [61]:
# Note: perhaps also parse units.  This should be done later. TODO
text_templates4 = \
    ["Is the $regulation for a property in the $zoning zoning district $regulation_value square feet?"]

name_templates4 = "template_dimreg_4var_yn_answer"

regulation_dict_t4 = {"minimum lot size":":minLotSize",}
zoning_t4 = ["R1a"]
regulation_value_t4 = [35_000]

sparql_template4 ="""
ASK {
		?zoning $regulation $regulation_value ;
                rdfs:label "${zoning}" .
}
"""

# expectated output from SPARQL: boolean (True/False)

# Iterators

In [40]:
import rdflib
uses_kg = rdflib.Graph()
# load the graph related to the permitted uses
uses_kg.parse("permits_use2.ttl")

# import rdflib
dimreq_kg = rdflib.Graph()
# load the graph related to the dimensional requirements
dimreq_kg.parse("bulk.ttl")

<Graph identifier=Ndaa275dededd400ab41a6a905f513452 (<class 'rdflib.graph.Graph'>)>

In [9]:
# SPARQL queries that are useful
class QueryUsesSparql():
    def __init__(self, uses_kg):
        self.uses_kg = uses_kg
        
    def all_uses_iter(self):
        """
        iterator of all the permitted uses in the knowledge graph
        """
        sparql = """
        SELECT ?use

        WHERE {
                ?zoning :permitsUse ?use .
        }
        """

        results = self.uses_kg.query(sparql)
        
        for use in set([str(res.use) for res in results]):
            yield use

    def all_zoning_iter(self):
        """
        iterator of all of the zoning districts in the knowledge graph
        """
        
        sparql = """
        SELECT ?zoning_label

        WHERE {
                ?zoning rdfs:label ?zoning_label .
        }
        """

        results = self.uses_kg.query(sparql)
        
        for zoning in set([str(res.zoning_label) for res in results]):
            yield zoning

    def all_uses_zoning_iter(self):
        """
        iterator of all the permitted uses in the knowledge graph
        
        returns tuple with (use, zoning)
        """
        sparql = """
        SELECT ?zoning_label ?use

        WHERE {
                ?zoning :permitsUse ?use .
                ?zoning rdfs:label ?zoning_label .
        }
        """

        results = self.uses_kg.query(sparql)
        
 #       for use in set([str(res.use) for res in results]):
        for res in results:
            yield res.use, res.zoning_label

In [10]:
# dimensinal regulations
# key is text, value is the name of predicate in the dimensional requirements KG
DIM_REGULATIONS = { "minimum lot size": ":minLotSize",
                    "maximum density": ":maxDensity",
                    "minimum lot width": ":minLotWidth",
                    "minimum lot depth": ":minLotDepth",
                    "minimum front setback": ":minFrontSetback",
                    "minimum side setback": ":minSideSetback",
                    "minimum rear setback": ":minRearSetback",
                    "maximum building height": ":maxBuildingHeight", }

class QueryDimensionsSparql():
    def __init__(self, dimensional_kg):
        self.dimensional_kg = dimensional_kg

    def all_zoning_iter(self):
        """
        iterator of all of the zoning districts in the knowledge graph
        """
        
        sparql = """
        SELECT ?zoning_label

        WHERE {
                ?zoning rdfs:label ?zoning_label .
        }
        """

        results = self.dimensional_kg.query(sparql)
        
        for zoning in set([str(res.zoning_label) for res in results]):
            yield zoning

### Testing Iterators - Permitted Uses code

In [38]:
qus = QueryUsesSparql(uses_kg)
for use in qus.all_uses_iter():
    print(use)

# works as intended

Minor automotive repair
family and group day care facilities
petroleum refining
research or testing laboratories
publicly owned and operated parks
furniture assembly
machine shops
mortuary and funeral homes
major automotive repair
woodworking shops
commercial printing and publishing
financial services
cotton gins
school and colleges operated for profit (including commercial, vocational and trade schools)
regional recycling center
physical fitness centers
schools and colleges (excluding colleges or trade schools operated for profit)
product distribution centers
rehabilitation centers
storage yards
salt works
amusement centers (including bowling alleys, golf driving ranges miniature golf courses, ice rinks, pool and billiard halls, and similar recreational uses)
libraries
liquid fertilizer manufacturing
private garages
small metal products design, casting, fabricating, and processing
public and governmental services
automotive sales
Dwellings
commercial bakeries
recreation centers
paper 

In [39]:
qus = QueryUsesSparql(uses_kg)
for zoning in qus.all_zoning_iter():
    print(zoning)

R2
C3
C2
FI2
A3
C1
C4
FI1
FI3
R1
R3
A2
A1


In [52]:
print("=== All Permitted uses with Zoning ===")
qus = QueryUsesSparql(uses_kg)
for use, zoning in qus.all_uses_zoning_iter():
    print(f"{use} -- {zoning}")

--- All Permitted uses with Zoning ---
Single-family dwellings -- R1
Single-family dwellings -- R2
Single-family dwellings -- R3
publicly owned and operated parks -- R1
publicly owned and operated parks -- R2
publicly owned and operated parks -- R3
recreation centers -- R1
recreation centers -- R2
recreation centers -- R3
swimming pools and playgrounds -- R1
swimming pools and playgrounds -- R2
swimming pools and playgrounds -- R3
police and fire department stations -- R1
police and fire department stations -- R2
police and fire department stations -- R3
police and fire department stations -- C1
police and fire department stations -- C2
police and fire department stations -- C3
police and fire department stations -- C4
public and governmental services -- R1
public and governmental services -- R2
public and governmental services -- R3
public and governmental services -- C1
public and governmental services -- C2
public and governmental services -- C3
public and governmental services -- C

### Testing -- Iterators -- Dimensional Requirements code

In [45]:
qdr = QueryDimensionsSparql(dimreq_kg)
for zoning in qdr.all_zoning_iter():
    print(zoning)

R1a
R3a
C3
R1b
R2b
C2
FI2
R1c
C4
C1
FI1
R1d
R2a
R3b
FI3
A3
A2
A1


# Template Generation

In [1]:
import rdflib
uses_kg = rdflib.Graph()
# load the graph related to the permitted uses
uses_kg.parse("permits_use2.ttl")

# import rdflib
dimreq_kg = rdflib.Graph()
# load the graph related to the permitted uses
dimreq_kg.parse("bulk.ttl")

<Graph identifier=N13d90bd1e6b240cca074e22a2fa93f97 (<class 'rdflib.graph.Graph'>)>

In [34]:
from string import Template

class TemplateGeneration():
    def __init__(self):
        self.templates = {}
        # Template 1 - template_use_1var_m_answer
        t1 = { 'template_name': 'template_use_1var_m_answer',
               'knowlege_graph': 'permitted_uses',
               'variables': ('use',),                # variables feed into the templates               
               'variable_names_sparql': ('zoning',), # variable resulting from SPARQL query
               'sparql_template': """
SELECT ?zoning_label

WHERE {
		?zoning :permitsUse "${use}" .
        ?zoning rdfs:label ?zoning_label .
}
""",
               'question_templates' : ["Which zoning districts allow ${use}?", 
                       "Which zoning districts permit ${use}?", 
                       "I would like to build ${use}.  Which zoning districts permits this use?"],
               'answer_datatype' : list,
 #              'iter_method' : 
        }
        self.templates['template_use_1var_m_answer'] = t1

        # Template 2 - template_use_2var_yn_answer
        t2 = { 'template_name': 'template_use_2var_yn_answer',
               'knowlege_graph': 'permitted_uses',
               'variables': ('use', 'zoning'),           # variables feed into the templates               
               'variable_names_sparql': tuple(),            # variable resulting from SPARQL query
               'sparql_template': """
ASK {
		?zoning :permitsUse "${use}" ;
                rdfs:label "${zoning}" .
}
""",
               'question_templates': ["Are $use allowed in a $zoning zoning district?",
                                      "Are $use permitted in a $zoning zoning district?"],
               'answer_datatype' : bool,
        }
        
        self.templates['template_use_2var_yn_answer'] = t2
        
        # Template 5 - template_use_1var_yn_answer
        t5 = { 'template_name': 'template_use_1var_yn_answer',
               'knowlege_graph': 'permitted_uses',
               'variables': ('use',),           # variables feed into the templates               
               'variable_names_sparql': tuple(),    # variable resulting from SPARQL query
               'sparql_template': """
ASK {
		?zoning :permitsUse "${use}" .
}
""",
               'question_templates': ["Are $use permitted?",
                                      "Are $use allowed?"],
               'answer_datatype' : bool,
        }

        self.templates['template_use_1var_yn_answer'] = t5

    def template_names(self):
        """template names"""
        return self.templates.keys()
    
    # NOTE: The dictionary was a design decision to aloow it to be easily extented
    # to add other variables.
    def generate_permitted_uses(self, use_kg, template_name):
        """
        generate the templates for the permitted uses
        use_kg - permitted uses knowlege graph,
        template_name - name of the template to use.
        
        return a generator that returns a dictionary.
        """
        if template_name not in self.templates:
            raise ValueError(f'template name: {template_name} does not exist.')
        
        if self.templates[template_name]['knowlege_graph'] != 'permitted_uses':
            # TODO better error type
            raise Error(f'template: {template_name}, is the wrong type of template, knowledge_graph must equal "permitted_uses"')

        # this is just used to get information about variables,
        # not to make queries of the KG.
        qus = QueryUsesSparql(uses_kg)

        iteratators = {
            ('use',) : qus.all_uses_iter(),
            ('use', 'zoning'): qus.all_uses_zoning_iter(),
            ('zoning',): qus.all_zoning_iter(),            
        }
        
        # variable_substitutions
        
        sparl_template = Template(self.templates[template_name]['sparql_template'])
        question_templates = [Template(q_tmpl) for q_tmpl in self.templates[template_name]['question_templates']]
        
        variable_names = self.templates[template_name]['variables']
        for q_template in question_templates:
            for variables in iteratators[variable_names]:
#                print("IT GETS THIS FAR")
#            for question_template in self.templates[template_name]['variables']
             #   print(f"variable_names: {variable_names}, variables: {variables}")
                if isinstance(variables, tuple):
                    variables_tuple = variables
                else:
                    variables_tuple = (variables,)
                varibs = dict(zip(variable_names, variables_tuple))
             #   print(f"varibs: {varibs}")
                result = {}
                result['sparql'] = sparl_template.substitute(varibs)
                result['question'] = q_template.substitute(varibs)
                yield result


In [35]:
tg = TemplateGeneration()

# tg.generate_permitted_uses(tg.template_names())
pu_iter = tg.generate_permitted_uses(uses_kg, 'template_use_1var_yn_answer')

In [36]:
for d in pu_iter:
    print(d)

{'sparql': '\nASK {\n\t\t?zoning :permitsUse "commercial printing and publishing" .\n}\n', 'question': 'Are commercial printing and publishing permitted?'}
{'sparql': '\nASK {\n\t\t?zoning :permitsUse "cotton gins" .\n}\n', 'question': 'Are cotton gins permitted?'}
{'sparql': '\nASK {\n\t\t?zoning :permitsUse "multiple family dwellings, such as apartment houses, boarding houses, condominiums and congregate residences" .\n}\n', 'question': 'Are multiple family dwellings, such as apartment houses, boarding houses, condominiums and congregate residences permitted?'}
{'sparql': '\nASK {\n\t\t?zoning :permitsUse "physical fitness centers" .\n}\n', 'question': 'Are physical fitness centers permitted?'}
{'sparql': '\nASK {\n\t\t?zoning :permitsUse "temporary buildings" .\n}\n', 'question': 'Are temporary buildings permitted?'}
{'sparql': '\nASK {\n\t\t?zoning :permitsUse "rehabilitation centers" .\n}\n', 'question': 'Are rehabilitation centers permitted?'}
{'sparql': '\nASK {\n\t\t?zoning :pe

In [37]:
tg = TemplateGeneration()

# tg.generate_permitted_uses(tg.template_names())
pu_iter = tg.generate_permitted_uses(uses_kg, 'template_use_1var_m_answer')

for d in pu_iter:
    print(d)

{'sparql': '\nSELECT ?zoning_label\n\nWHERE {\n\t\t?zoning :permitsUse "commercial printing and publishing" .\n        ?zoning rdfs:label ?zoning_label .\n}\n', 'question': 'Which zoning districts allow commercial printing and publishing?'}
{'sparql': '\nSELECT ?zoning_label\n\nWHERE {\n\t\t?zoning :permitsUse "cotton gins" .\n        ?zoning rdfs:label ?zoning_label .\n}\n', 'question': 'Which zoning districts allow cotton gins?'}
{'sparql': '\nSELECT ?zoning_label\n\nWHERE {\n\t\t?zoning :permitsUse "multiple family dwellings, such as apartment houses, boarding houses, condominiums and congregate residences" .\n        ?zoning rdfs:label ?zoning_label .\n}\n', 'question': 'Which zoning districts allow multiple family dwellings, such as apartment houses, boarding houses, condominiums and congregate residences?'}
{'sparql': '\nSELECT ?zoning_label\n\nWHERE {\n\t\t?zoning :permitsUse "physical fitness centers" .\n        ?zoning rdfs:label ?zoning_label .\n}\n', 'question': 'Which zonin

# Get properties from the Knowledge Graph.

In [7]:
import rdflib
dimreq_kg = rdflib.Graph()
# load the graph related to the dimensional requirements
dimreq_kg.parse("bulk2.ttl")

sparql_properties = """
SELECT ?property_name ?property_label

WHERE {
    ?property_name a rdf:Property ;
                rdf:label  ?property_label .
}
"""

results = dimreq_kg.query(sparql_properties)

for row in results:
    print(f'Name {row.property_name}, Label "{row.property_label}"')

Name http://www.example.org/ns/lu/zoning#minLotSize, Label "minimum lot size"
Name http://www.example.org/ns/lu/zoning#maxDensity, Label "maximum density"
Name http://www.example.org/ns/lu/zoning#minLotWidth, Label "minimum lot width"
Name http://www.example.org/ns/lu/zoning#minLotDepth, Label "minimum lot depth"
Name http://www.example.org/ns/lu/zoning#minFrontSetback, Label "minimum front setback"
Name http://www.example.org/ns/lu/zoning#minSideSetback, Label "minimum side setback"
Name http://www.example.org/ns/lu/zoning#minRearSetback, Label "minimum rear setback"
Name http://www.example.org/ns/lu/zoning#maxBuildingHeight, Label "maximumBuildingHeight"


In [9]:
for row in results:
    print(f'Name {row.property_name.fragment}, Label "{row.property_label}"')

Name minLotSize, Label "minimum lot size"
Name maxDensity, Label "maximum density"
Name minLotWidth, Label "minimum lot width"
Name minLotDepth, Label "minimum lot depth"
Name minFrontSetback, Label "minimum front setback"
Name minSideSetback, Label "minimum side setback"
Name minRearSetback, Label "minimum rear setback"
Name maxBuildingHeight, Label "maximumBuildingHeight"


In [17]:
{ str(row.property_label): ':'+row.property_name.fragment for row in results }

{'minimum lot size': ':minLotSize',
 'maximum density': ':maxDensity',
 'minimum lot width': ':minLotWidth',
 'minimum lot depth': ':minLotDepth',
 'minimum front setback': ':minFrontSetback',
 'minimum side setback': ':minSideSetback',
 'minimum rear setback': ':minRearSetback',
 'maximumBuildingHeight': ':maxBuildingHeight'}

# SCRATCH

In [2]:
sparql = """
SELECT ?use

WHERE {
        ?zoning :permitsUse ?use .
}
"""

results = uses_kg.query(sparql)


In [27]:
for r in set([str(res.use) for res in results]):
    print(r)

Minor automotive repair
family and group day care facilities
petroleum refining
research or testing laboratories
publicly owned and operated parks
furniture assembly
machine shops
mortuary and funeral homes
major automotive repair
woodworking shops
commercial printing and publishing
financial services
cotton gins
school and colleges operated for profit (including commercial, vocational and trade schools)
regional recycling center
physical fitness centers
schools and colleges (excluding colleges or trade schools operated for profit)
product distribution centers
rehabilitation centers
storage yards
salt works
amusement centers (including bowling alleys, golf driving ranges miniature golf courses, ice rinks, pool and billiard halls, and similar recreational uses)
libraries
liquid fertilizer manufacturing
private garages
small metal products design, casting, fabricating, and processing
public and governmental services
automotive sales
Dwellings
commercial bakeries
recreation centers
paper 

In [4]:
print(next(results))

TypeError: 'SPARQLResult' object is not an iterator

In [ ]:
for res in results

{'Agricultural Use',
 'Auto-dismantling yards',
 'Dwellings',
 'Maintenance/storage buildings',
 'Minor automotive repair',
 'Open space',
 'Self-storage warehouses',
 'Single-family dwellings',
 'Stadiums and arenas',
 'Two-family dwellings',
 'Two-family dwellingsmultiple family dwellings, such as apartment houses, boarding houses, condominiums and congregate residences',
 'alcohol manufacturing',
 'amusement centers (including bowling alleys, golf driving ranges miniature golf courses, ice rinks, pool and billiard halls, and similar recreational uses)',
 'auto body shops',
 'automotive motor fuel dispensing facilities',
 'automotive sales',
 'automotive self-service motor fuel dispensing facilities',
 'boat storage yards',
 'breweries',
 'building material supply sales (wholesale and retail)',
 'buildings accessory to the permitted used (including private garages and accessory living quarters)',
 'business services',
 'carpet manufacturing',
 'commercial bakeries',
 'commercial prin

In [5]:
results.type

['__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_bindings',
 '_genbindings',
 '_get_bindings',
 '_set_bindings',
 'askAnswer',
 'bindings',
 'graph',
 'parse',
 'serialize',
 'type',
 'vars']

In [6]:
results.type

'SELECT'

In [8]:
results.vars

[rdflib.term.Variable('use')]

In [9]:
results.serialize

<bound method Result.serialize of <rdflib.plugins.sparql.processor.SPARQLResult object at 0x7fb7a8637190>>

In [10]:
type(results.serialize)

method

In [12]:
# results.serialize()
results.bindings

[{rdflib.term.Variable('use'): rdflib.term.Literal('Single-family dwellings')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('Single-family dwellings')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('Single-family dwellings')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('publicly owned and operated parks')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('publicly owned and operated parks')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('publicly owned and operated parks')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('recreation centers')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('recreation centers')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('recreation centers')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('swimming pools and playgrounds')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('swimming pools and playgrounds')},
 {rdflib.term.Variable('use'): rdflib.term.Literal('swimming pools and playgrounds')},
 {rd

In [23]:
results.bindings[2].keys()

KeysView({rdflib.term.Variable('use'): rdflib.term.Literal('Single-family dwellings')})